In [2]:
import sys
from copy import deepcopy
import tqdm
from tqdm import tqdm

import torch
import torch.nn as nn
from torchvision import models
import torch.jit
from copy import deepcopy

import torch
import torch.nn as nn
import argparse
import os
import sys
import logging
import time
import random
import torch
import numpy as np
from datetime import datetime
from iopath.common.file_io import g_pathmgr
from yacs.config import CfgNode as CfgNode
import logging

import torch
import torch.optim as optim

# from robustbench.data import load_cifar10c
# from robustbench.model_zoo.enums import ThreatModel
# from robustbench.utils import load_model
# from robustbench.utils import clean_accuracy as accuracy

import tent
import norm

from conf import cfg, load_cfg_fom_args

/home/haileyq/jupyter2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **LOAD THE MODEL**

In [3]:
# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the ResNet50 model pre-trained on ImageNet
conv_base = models.resnet50(pretrained=False)

# Customize the fully connected layer according to your model's architecture
num_ftrs = conv_base.fc.in_features  # Get number of features in last layer
conv_base.fc = nn.Sequential(  # Replace the fully connected layer
    nn.Flatten(),
    nn.Linear(num_ftrs, 256),
    nn.ReLU(),
    nn.Linear(256, 3),
    nn.Softmax(dim=1)
)

conv_base = conv_base.to(device)  # Move model to GPU if available

/home/haileyq/jupyter2/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/haileyq/jupyter2/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
model_path = '../TvN_350_SN_D256_Initial_Ep7_fullmodel.pth'
conv_base = torch.load(model_path, map_location=device)

In [5]:
logger = logging.getLogger(__name__)
m_p_s = 350

### **STAIN NORMALIZATION**

In [6]:
import staintools
import os
from PIL import Image
from torchvision import transforms

In [7]:
# Standartization image and Stain normalization setup
def setup_stain_normalizer(standard_img_path):
    st = staintools.read_image(standard_img_path)
    st = staintools.LuminosityStandardizer.standardize(st)
    stain_norm = staintools.StainNormalizer(method='macenko')
    stain_norm.fit(st)
    return stain_norm

def process_and_standardize_image(image_path, stain_norm):
    # Load, optionally standardize brightness, and apply stain normalization
    image = staintools.read_image(image_path)
    image = staintools.LuminosityStandardizer.standardize(image)
    normalized_image = stain_norm.transform(image)
    return normalized_image

# Transformation pipeline
transform_pipeline = transforms.Compose([
    transforms.Resize((m_p_s, m_p_s)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### **PREPROCESS DATASET**

In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

def preprocess_dataset(data_dir, stain_norm, transform_pipeline, batch_size):
    # Setup the ImageFolder dataset
    dataset = datasets.ImageFolder(root=data_dir, transform=transforms.Compose([
        transforms.Lambda(lambda x: Image.fromarray(staintools.LuminosityStandardizer.standardize(np.array(x)))),
        transforms.Lambda(lambda x: Image.fromarray(stain_norm.transform(np.array(x)))),
        transform_pipeline 
    ]))
    
    # DataLoader for batching
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return data_loader

### **EVALUATE**

In [9]:
# def evaluate(description, transform_pipeline, base_model):
#     load_cfg_fom_args(description)
#     # Configure model
#     if cfg.MODEL.ADAPTATION == "source":
#         logger.info("test-time adaptation: NONE")
#         model = setup_source(base_model)
#     if cfg.MODEL.ADAPTATION == "norm":
#         logger.info("test-time adaptation: NORM")
#         model = setup_norm(base_model)
#     if cfg.MODEL.ADAPTATION == "tent":
#         logger.info("test-time adaptation: TENT")
#         model = setup_tent(base_model)
#     # Setup stain normalization and transformation pipeline as before
#     stain_norm = setup_stain_normalizer("projects/def-msh-ab/haileyq/tta/tent/standard_he_stain_small.jpg")
#     transform_pipeline = transform_pipeline
#     data_loader = preprocess_dataset(cfg.DATA_DIR, stain_norm, transform_pipeline, cfg.TEST.BATCH_SIZE)

#     total_correct = 0
#     total_images = 0
#     model.eval()  # Ensure the model is in evaluation mode

#     # Evaluation loop over batches from DataLoader
#     for inputs, labels in data_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         total_correct += torch.sum(preds == labels.data)
#         total_images += inputs.size(0)
    
#     accuracy = total_correct.double() / total_images
#     logger.info(f'Model accuracy on new dataset: {accuracy:.4f}')

In [10]:
def evaluate(data_loader, tent_adapter):
    print("Starting evaluation...")
    logging.info("Starting evaluation...")
    tent_adapter.model.eval()  # Set the underlying model to evaluation mode
    total_correct = 0
    total_images = 0
    start_time = time.time()

    with torch.no_grad():
        # Enumerate over the data_loader to get both index and data
        for batch_idx, (inputs, labels) in enumerate(tqdm(data_loader, desc="Evaluating")):
            batch_start = time.time()  # Record the start time of the batch processing
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = tent_adapter(inputs)  # Use tent_adapter for forward pass
            _, preds = torch.max(outputs, 1)
            total_correct += torch.sum(preds == labels.data)
            total_images += inputs.size(0)
            batch_end = time.time()  # Record the end time of the batch processing
            
            # Print and log batch processing info
            print(f"Processed batch {batch_idx + 1}/{len(data_loader)}. Batch accuracy: {torch.sum(preds == labels.data).item() / inputs.size(0):.4f}")
            logging.info(f"Processed batch {batch_idx + 1}/{len(data_loader)}. Batch accuracy: {torch.sum(preds == labels.data).item() / inputs.size(0):.4f}. Time taken: {batch_end - batch_start:.2f}s")
    
    accuracy = total_correct.double() / total_images
    end_time = time.time()
    print(f"Finished evaluation. Total accuracy: {accuracy:.4f}. Total time: {end_time - start_time:.2f}s")
    logging.info(f'Finished evaluation. Model accuracy on new dataset: {accuracy:.4f}. Total time: {end_time - start_time:.2f}s')

### **RUN TTA**

In [11]:
stain_norm = setup_stain_normalizer("standard_he_stain_small.jpg")
cfg.merge_from_file("cfgs/tent.yaml")
print(cfg.MODEL.ADAPTATION) 

tent


In [12]:
data_loader = preprocess_dataset(cfg.DATA_DIR, stain_norm, transform_pipeline, cfg.TEST.BATCH_SIZE)

In [13]:
model = tent.configure_model(conv_base)
params, _ = tent.collect_params(model)
optimizer = torch.optim.Adam(params)  # Customize optimizer as needed

# Initialize the Tent adapter
tent_adapter = tent.Tent(model=model, optimizer=optimizer, steps=1, episodic=True)

In [14]:
evaluate(data_loader=data_loader, tent_adapter=tent_adapter)

Starting evaluation...


Evaluating:   0%|          | 1/938 [00:08<2:14:12,  8.59s/it]

Processed batch 1/938. Batch accuracy: 0.4297


Evaluating:   0%|          | 2/938 [00:15<1:55:38,  7.41s/it]

Processed batch 2/938. Batch accuracy: 0.4453


...

Processed batch 936/938. Batch accuracy: 0.5625


Evaluating: 100%|█████████▉| 937/938 [2:58:10<00:12, 12.46s/it]

Processed batch 937/938. Batch accuracy: 0.5156


Evaluating: 100%|██████████| 938/938 [2:58:16<00:00, 11.40s/it]

Processed batch 938/938. Batch accuracy: 0.5312
Finished evaluation. Total accuracy: 0.4090. Total time: 10696.42s
